# PREPROCESSING PIPELINE

**Any dataset preprocessing will happen here**

In [1]:
# Run necessary imports here

import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor

# Parsing residential sales data and querying GEOID

## THIS TOOK ~115 MINUTES TO COMPLETE, DO NOT RUN UNLESS NECESSARY!

In [ ]:
fp1 = "chicago_residential_sales_data.csv"

table1 = pd.read_csv(fp1)

# property class codes for regression, check dataset documentation for more info in `DETAIL.md`
reg_class = set([200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 234, 278, 295])    
nonreg_idx = table1[~table1['Property Class'].isin(reg_class)].index

coords_label = ['Longitude', 'Latitude']

table1 = table1.drop(nonreg_idx)            # drop non-regression classes
table1 = table1.dropna(subset=coords_label) # drop NaN if lat or lon values are NaN

coords = table1[coords_label].values

url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"

params = {
    'benchmark': 4,
    'vintage':4,
    'layers': 8,
    'format': 'json'}

def fetch_census_tract(coord):
    """Fetch Census Tract GEOID for a given coordinate."""
    x, y = coord
    try:
        response = requests.get(url, params={**params, 'x': x, 'y': y}).json()
        return response['result']['geographies']['Census Tracts'][0]['GEOID']
    except Exception:
        return None  # Handle errors gracefully

# THREADING THE GEOID QUERYING, UNCOMMENT IF NEEDED
# with ThreadPoolExecutor(max_workers=20) as executor:
#     census_tracts = list(executor.map(fetch_census_tract, coords))

# table1['GEOID'] = census_tracts
# table1.to_csv("updt_chicago_residential_sales.csv")

# display(pd.read_csv("updt_chicago_residential_sales.csv"))

,Unnamed: 0,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,...,Lot Size Squared,Improvement Size Squared,Pure Market Filter,Garage Indicator,Neigborhood Code (mapping),Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood,GEOID
0,2,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,...,2.025000e+07,55279225.0,1,1.0,13,67.082039,9.327379,86.226446,7713,1.703125e+10
1,5,4252000820000,204,100,33898.0,25,5.0,0.0,2.0,4.0,...,1.149074e+09,22014864.0,1,1.0,100,184.114095,7.810250,68.498175,25100,1.703180e+10
2,8,14322110150000,208,12,3720.0,74,3.0,0.0,2.0,6.0,...,1.383840e+07,18378369.0,1,1.0,12,60.991803,2.000000,65.475186,7412,1.703107e+10
3,9,27021200080000,204,34,16079.0,28,1.0,0.0,3.0,1.0,...,2.585342e+08,4875264.0,1,1.0,34,126.802997,5.385165,46.989360,2834,1.703182e+10
4,10,1191010060000,204,21,220766.0,10,1.0,0.0,2.0,4.0,...,4.873763e+10,16605625.0,0,1.0,21,469.857425,5.000000,63.835727,1021,1.703180e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406624,583357,15292270020000,203,104,8267.0,31,1.0,0.0,3.0,1.0,...,6.834329e+07,1261129.0,1,1.0,104,90.923044,7.416198,33.511192,31104,1.703182e+10
406625,583360,20203200240000,234,171,3125.0,72,4.0,0.0,2.0,1.0,...,9.765625e+06,938961.0,0,0.0,171,55.901699,6.782330,31.128765,72171,1.703183e+10
406626,583366,20021110150000,205,10,2750.0,70,2.0,0.0,2.0,1.0,...,7.562500e+06,4588164.0,1,0.0,10,52.440442,10.954451,46.281746,7010,1.703184e+10
406627,583367,20021110150000,205,10,2750.0,70,2.0,0.0,2.0,1.0,...,7.562500e+06,4588164.0,1,0.0,10,52.440442,10.954451,46.281746,7010,1.703184e+10


# Re-query missing values caused by timeouts or other exceptions during API requests and update table

In [3]:
fp1 = "updt_chicago_residential_sales.csv"
table = pd.read_csv(fp1)

coords_label = ['Longitude', 'Latitude']

nulls = table.loc[table['GEOID'].isnull()].copy()
# display(nulls[coords_label])

url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
params = {
    'benchmark': 4,
    'vintage': 4,
    'layers': 8,
    'format': 'json'
}

def fetch_census_tract(row):
    """Fetch Census Tract GEOID for a given row's coordinates."""
    x, y = row['Longitude'], row['Latitude']
    try:
        response = requests.get(url, params={**params, 'x': x, 'y': y}).json()
        return response['result']['geographies']['Census Tracts'][0]['GEOID']
    except Exception:
        return None  # Return None if the request fails

# Use threading to speed up API calls
with ThreadPoolExecutor(max_workers=10) as executor:
    updated_geoids = list(executor.map(fetch_census_tract, nulls.to_dict(orient="records")))

# Update the GEOID column in the original table
table.loc[table['GEOID'].isnull(), 'GEOID'] = updated_geoids
# display(updated_geoids)

table.to_csv("geoid_chicago_residential_sales.csv")

C:\Users\bloon\AppData\Local\Temp\ipykernel_15212\3970605315.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['17031830800', '17031823604', '17031805402', '17031740400', '17031661000', '17031661000', '17031827300', '17031730600', '17031805501', '17031660900', '17031031300', '17031820800', '17031830003', '17031842600', '17031822400', '17031821900', '17031670300', '17031818500', '17031804313', '17031801702', '17031818500', '17031661100', '17031803500', '17031050900', '17031828601', '17031200200', '17031810800', '17031830600', '17031804306', '17031828100', '17031691400', '17031420700', '17031813400', '17031801608', '17031804306', '17031805902', '17031040201', '17031803500', '17031822601', '17031560800', '17031060300', '17031830004', '17031843400', '17031825303', '17031819000', '17031150402', '17031820101', '17031819600', '17031822302', '17031825503', '17031803605', '17031832600', '17031220400', '170312